## Insert Parameters
Parameters can be inserted automatically into a Copasi model.

### Build a demonistration model
While antimony or the COPASI user interface are the preferred ways to build a model, PyCoTools does have a mechanism for constructing COPASI models. For variation and demonstration, this method is used here.

In [4]:
import os
import site
site.addsitedir('D:\pycotools3')
from pycotools3 import model, tasks, viz
## Choose a working directory for model
working_directory = os.path.abspath('')
copasi_file = os.path.join(working_directory, 'MichaelisMenten.cps')

if os.path.isfile(copasi_file):
    os.remove(copasi_file)

    
kf = 0.01
kb = 0.1
kcat = 0.05
with model.Build(copasi_file) as m:
    m.name = 'Michaelis-Menten'
    m.add('compartment', name='Cell')
    
    m.add('metabolite', name='P', concentration=0)
    m.add('metabolite', name='S', concentration=30)
    m.add('metabolite', name='E', concentration=10)
    m.add('metabolite', name='ES', concentration=0)
    
    m.add('reaction', name='S bind E', expression='S + E -> ES', rate_law='kf*S*E',
          parameter_values={'kf': kf})
    
    m.add('reaction', name='S unbind E', expression='ES -> S + E', rate_law='kb*ES',
         parameter_values={'kb': kb})

    m.add('reaction', name='ES produce P', expression='ES -> P + E', rate_law='kcat*ES', 
          parameter_values={'kcat': kcat})
    
mm = model.Model(copasi_file)
mm

Model(name=Michaelis-Menten, time_unit=s, volume_unit=ml, quantity_unit=mmol)

### Insert Parameters from Python Dictionary

In [5]:
params = {'E': 100,
          'P': 150}

## Insert into model 
I = model.InsertParameters(mm, parameter_dict=params)
##format the parameters for displaying nicely
I.parameters.index = ['Parameter Value']
I.parameters.transpose()

,Parameter Value
E,100
P,150


Alternatively use `inplace=True` argument (analogous to the `pandas` library) to modify the object inplace, rather than needing to assign

In [6]:
model.InsertParameters(mm, parameter_dict=params, inplace=True)

### Insert Parameters from Pandas DataFrame

In [7]:
import pandas
params = {'(S bind E).kf': 50,
          '(S unbind E).kb': 96}
df = pandas.DataFrame(params, index=[0])
df

,(S bind E).kf,(S unbind E).kb
0,50,96


In [8]:
model.InsertParameters(mm, df=df, inplace=True)

### Insert Parameters from Parameter Estimation Output
First we'll get some parameter estimation data by [fitting](ParameterEstimation.ipynb) a model to [simulated]('Timecourse.ipynb') data. 

In [9]:
fname = os.path.join(os.path.abspath(''), 'timecourse.txt')
data = mm.simulate(0, 50, 1, report_name=fname)
assert os.path.isfile(fname)

In [10]:
with  tasks.ParameterEstimation.Context(copasi_file, fname, context='s', parameters='l') as context:
    context.randomize_start_values = True
    context.lower_bound = 0.01
    context.upper_bound = 100
    context.run_mode = True
    config = context.get_config()
print(config)
PE = tasks.ParameterEstimation(config)

datasets:
    experiments:
        timecourse:
            affected_models:
            - MichaelisMenten
            filename: D:\pycotools3\docs\source\Tutorials\timecourse.txt
            mappings:
                E:
                    model_object: E
                    object_type: Metabolite
                    role: dependent
                ES:
                    model_object: ES
                    object_type: Metabolite
                    role: dependent
                P:
                    model_object: P
                    object_type: Metabolite
                    role: dependent
                S:
                    model_object: S
                    object_type: Metabolite
                    role: dependent
                Time:
                    model_object: Time
                    role: time
            normalize_weights_per_experiment: true
            separator: "\t"
    validations: {}
items:
    fit_items:
        (ES produce P).kcat:
            aff

In [12]:
mm

Model(name=Michaelis-Menten, time_unit=s, volume_unit=ml, quantity_unit=mmol)

Now we can insert the estimated parameters using: 

In [13]:
##index=0 for best parameter set (i.e. lowest RSS)
model.InsertParameters(mm, parameter_path=PE.results_directory, index=0, inplace=True) 

InputError: {'MichaelisMenten': 'D:\\pycotools3\\docs\\source\\Tutorials\\Problem1\\Fit1\\MichaelisMenten\\ParameterEstimationData'} not in [<class 'pycotools3.tasks.TimeCourse'>, <class 'pycotools3.tasks.Scan'>, <class 'pycotools3.tasks.ParameterEstimation'>, <class 'str'>, <class 'pycotools3.viz.Parse'>, <class 'pandas.core.frame.DataFrame'>]

### Insert Parameters using the `model.Model().insert_parameters` method
The same means of inserting parameters can be used from the model object itself

In [ ]:
mm.insert_parameters(parameter_dict=params, inplace=True)

### Change parameters using `model.Model().set`
Individual parameters can also be changed using the `set` method. For example, we could set the `metabolite` with name `S` `concentration` or `particle numbers` to `55`

In [ ]:
mm.set('metabolite', 'S', 55, 'name', 'concentration')

## or 

mm.set('metabolite', 'S', 55, 'name', 'particle_numbers')